### 结合websocket和thread，构造一个从ws获取实时数据，并保存到deque的线程
实际上结合了《websocket.ipynb》和《queue_deque_threading.ipynb》，构造了一个能够“独立地接收数据”的线程，而主线程可以基于收到的数据进行进一步地处理、分析、交易等。

In [3]:
from threading import Thread
from collections import deque
import websocket
import json
import time
import copy
import numpy as np


# 数据接收的类
class Data_Receiver(Thread):
    '''
    Data_Receiver类：
    一个多线程的类，给定market/symbol/data参数后，生成一个cache，这个数据存储了maxlen个从websocke接收到的数据，
    其中最后的一个数据为最新的数据，而maxlen以前的数据则被pop了（用到了collections的deque数据类型）。这个多线程被start后，
    每次需要时进行list(Data_Receiver.cache)即可获得所需数据的列表。
    
    Input:
        maxlen: 队列的最大长度
        ws_url: websocket的地址名
        data: 需要的具体数据名
    
    Notices:
        只是一个简单的例子，这里的data确定了如何访问返回的json数据内容，应该有更加通用而灵活的办法
    '''
    def __init__(self, maxlen=None, ws_url=None, data=None):
        super(Data_Receiver, self).__init__()
        self._cache = deque(maxlen=maxlen)
        self.ws_url = ws_url
        self.data_from_json = "msg_json" + data
    
    @property
    def cache(self):
        return copy.deepcopy(self._cache)
    
    # on_回调函数，websocket客户端使用的函数，注意这里第一个参数是self
    def on_message(self, message):
        msg_json = json.loads(message)
        data = eval(self.data_from_json)  # eval()函数，执行一个字符串
        self._cache.append(data)
    
    def on_error(ws, error):
        print(error)
    
    def on_close(ws):
        print("### closed ###")
    
    def on_open(ws):
        print("### open ###")

    def append_data(self):
        ws = websocket.WebSocketApp(self.ws_url,
                                  on_message = self.on_message,
                                  on_error = self.on_error,
                                  on_close = self.on_close)
        ws.on_open = self.on_open
        while True:  # 一旦on_close触发跳出run_forever，则立马再次进入run_forever(while True)
            try:
                ws.run_forever(ping_timeout=10)
            except:
                pass

    def run(self):
        self.append_data()

In [4]:
# 使用上述数据接收的类
### start
binance_btcusdt_trade_ws_url = "wss://stream.binance.com:9443/ws/btcusdt@trade"
binance_btcusdt_trade = Data_Receiver(maxlen=5, ws_url=binance_btcusdt_trade_ws_url, data="['p']")
binance_btcusdt_trade.start()
for n in range(10):
    time.sleep(40*np.random.rand(1))
    binance_btcusdt_trade_list = list(binance_btcusdt_trade.cache)
    binance_btcusdt_trade_list_len = len(binance_btcusdt_trade_list)
    print(f'--get queue data {n+1} times: {binance_btcusdt_trade_list}--')
    print(f'lastest data of queue: {binance_btcusdt_trade_list[binance_btcusdt_trade_list_len-1]}')

### open ###
--get queue data 1 times: ['3688.65000000', '3688.09000000', '3687.29000000', '3687.26000000', '3687.23000000']--
lastest data of queue: 3687.23000000
--get queue data 2 times: ['3688.65000000', '3688.65000000', '3688.65000000', '3687.35000000', '3687.35000000']--
lastest data of queue: 3687.35000000
--get queue data 3 times: ['3688.02000000', '3688.02000000', '3688.02000000', '3688.23000000', '3688.03000000']--
lastest data of queue: 3688.03000000
--get queue data 4 times: ['3688.85000000', '3688.79000000', '3688.79000000', '3688.05000000', '3688.05000000']--
lastest data of queue: 3688.05000000
--get queue data 5 times: ['3687.92000000', '3688.00000000', '3687.72000000', '3688.00000000', '3687.98000000']--
lastest data of queue: 3687.98000000
--get queue data 6 times: ['3687.92000000', '3688.00000000', '3687.72000000', '3688.00000000', '3687.98000000']--
lastest data of queue: 3687.98000000
--get queue data 7 times: ['3687.99000000', '3687.99000000', '3688.00000000', '36

注意到，这里是将websocket的相关内容放在数据接收类里面，而on_message()的第一个参数是self（为了能够在这里面将数据append到deque上），但是websocket包的[官方例子](https://pypi.org/project/websocket-client/)表示该回调函数的参数应该是如下形式"on_message(ws, message)"。那么，为什么这里的ws可以被self这个参数替代呢？ <br/>
查看websocket._app的源码，最后的回调过程如下： <br/>
```
def _callback(self, callback, *args):
    if callback:
        try:
            if inspect.ismethod(callback):
                callback(*args)
            else:
                callback(self, *args)
```
可以看到，当传入的函数callback是一个实例方法时（bounded method），可以在给函数定义时的第一个参数设定为self，这样调用的情况“callback(*args)”相当于“on_message(data)”，不再需要传入ws。而如果不是一个实例方法时（比如一个函数），则代码会自动补上一个参数self（这里实际上就是ws）。